In [7]:
import numpy as np
import scipy as sp
from scipy import linalg
from scipy import optimize
from scipy import interpolate
import sympy as sm

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

# Optimization problem 1

# Steady state

## Analytical solution

First we define all **symbols**:

In [11]:
k = sm.symbols('k')
h = sm.symbols('h')
alpha = sm.symbols('alpha')
delta = sm.symbols('delta')
phi = sm.symbols ('phi')
s = sm.symbols('s')
g = sm.symbols('g')
n = sm.symbols('n')
ng = sm.symbols('ng')
Sk = sm.symbols('sk')
Sh = sm.symbols ('sh')

# Conclusion